# Homework 6

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
import numpy as np
import os
import tempfile
import pdb


BLOCKS_DOMAIN = """(define (domain prodigy-bw)
  (:requirements :strips)
  (:predicates (on ?x ?y)
	       (on-table ?x)
	       (clear ?x)
	       (arm-empty)
	       (holding ?x)
	       )
  (:action pick-up
	     :parameters (?ob1)
	     :precondition (and (clear ?ob1) (on-table ?ob1) (arm-empty))
	     :effect
	     (and (not (on-table ?ob1))
		   (not (clear ?ob1))
		   (not (arm-empty))
		   (holding ?ob1)))
  (:action put-down
	     :parameters (?ob)
	     :precondition (holding ?ob)
	     :effect
	     (and (not (holding ?ob))
		   (clear ?ob)
		   (arm-empty)
		   (on-table ?ob)))
  (:action stack
	     :parameters (?sob ?sunderob)
	     :precondition (and (holding ?sob) (clear ?sunderob))
	     :effect
	     (and (not (holding ?sob))
		   (not (clear ?sunderob))
		   (clear ?sob)
		   (arm-empty)
		   (on ?sob ?sunderob)))
  (:action unstack
	     :parameters (?sob ?sunderob)
	     :precondition (and (on ?sob ?sunderob) (clear ?sob) (arm-empty))
	     :effect
	     (and (holding ?sob)
		   (clear ?sunderob)
		   (not (clear ?sob))
		   (not (arm-empty))
		   (not (on ?sob ?sunderob)))))
"""

BLOCKS_PROBLEM_1 = """(define (problem bw-simple)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (clear a) (arm-empty) (on a b) (on-table b))
  (:goal (and (on-table a) (clear b))))
"""

BLOCKS_PROBLEM_2 = """(define (problem bw-sussman)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (on-table a) (on-table b) (on c a)
		(clear b) (clear c) (arm-empty))
  (:goal (and (on a b) (on b c))))
"""

BLOCKS_PROBLEM_3 = """(define (problem bw-large-a)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9)
  (:init (arm-empty)
	 (on 3 2)
	 (on 2 1)
	 (on-table 1)
	 (on 5 4)
	 (on-table 4)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (on-table 6)
	 (clear 3)
	 (clear 5)
	 (clear 9))
  (:goal (and
	  (on 1 5)
	  (on-table 5)
	  (on 8 9)
	  (on 9 4)
	  (on-table 4)
	  (on 2 3)
	  (on 3 7)
	  (on 7 6)
	  (on-table 6)
	  (clear 1)
	  (clear 8)
	  (clear 2)
	  )))
"""

BLOCKS_PROBLEM_4 = """(define (problem bw-large-b)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11)
  (:init (arm-empty)
	 (on 3 2)
	 (on 2 1)
	 (on-table 1)
	 (on 11 10)
	 (on 10 5)
	 (on 5 4)
	 (on-table 4)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (on-table 6)
	 (clear 3)
	 (clear 11)
	 (clear 9))
  (:goal (and
	  (on 1 5)
	  (on 5 10)
	  (on-table 10)
	  (on 8 9)
	  (on 9 4)
	  (on-table 4)
	  (on 2 3)
	  (on 3 11)
	  (on 11 7)
	  (on 7 6)
	  (on-table 6)
	  (clear 1)
	  (clear 8)
	  (clear 2)
	  )))
"""

def get_task_definition_str(domain_pddl_str, problem_pddl_str):
  """Get pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the contents of a domain.pddl file.
    problem_pddl_str: A str, the contents of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  domain_file = tempfile.NamedTemporaryFile(delete=False)
  problem_file = tempfile.NamedTemporaryFile(delete=False)
  with open(domain_file.name, 'w') as f:
    f.write(domain_pddl_str)
  with open(problem_file.name, 'w') as f:
    f.write(problem_pddl_str)
  parser = Parser(domain_file.name, problem_file.name)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)
  os.remove(domain_file.name)
  os.remove(problem_file.name)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def get_task_definition(domain_pddl, problem_pddl):
  """Get pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the name of a domain.pddl file.
    problem_pddl_str: A str, the name of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  parser = Parser(domain_pddl, problem_pddl)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def h_add_test(prob_str, expected_h):
  task = get_task_definition_str(BLOCKS_DOMAIN, prob_str)
  h = h_add(task)
  assert h == expected_h, f"Expected h={expected_h}, but got {h}."
  return h

def h_ff_test(prob_str, expected_h):
  task = get_task_definition_str(BLOCKS_DOMAIN, prob_str)
  h = h_ff(task)
  assert h == expected_h, f"Expected h={expected_h}, but got {h}."




## Problems

### Construct Reduced Planning Graph (RPG)
Given a Pyperplan Task instance and an optional state, return (F_t, A_t), two lists as defined in the RPG pseudo-code from lecture.

For reference, our solution is **13** lines of code.

In [ ]:
def construct_rpg(task, state=None):
  """Constructs a Relaxed Planning Graph (RPG) for a given Task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a Pyperplan Task instance
    state: a set of facts

  Returns:
    rpg: F_sets (ordered list of fact sets), A_sets (ordered list of operator sets)
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def construct_rpg_test1():
  task = get_task_definition_str(BLOCKS_DOMAIN, BLOCKS_PROBLEM_1)
  expected_F_sets = [{'(clear a)', '(on a b)', '(on-table b)', '(arm-empty)'}, {'(on-table b)', '(arm-empty)', '(clear a)', '(on a b)', '(holding a)', '(clear b)'}, {'(arm-empty)', '(on-table a)', '(holding a)', '(holding b)', '(on-table b)', '(on a a)', '(clear a)', '(on a b)', '(clear b)'}]
  F_sets, A_sets = construct_rpg(task)
  assert len(F_sets) == len(expected_F_sets), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
  for F_set, expected_F_set in zip(F_sets, expected_F_sets):
      assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
  expected_A_set_strs = [{'(unstack a b)'}, {'(unstack a b)', '(pick-up b)', '(stack a a)', '(stack a b)', '(put-down a)'}]
  assert len(A_sets) == len(expected_A_set_strs), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
  for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
      A_set_str = {a.name for a in A_set}
      assert A_set_str == expected_A_set_str, f"Incorrect A_set: {A_set_str}, expected {expected_A_set_str}"
  return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test1()
def construct_rpg_test2():
  task = get_task_definition_str(BLOCKS_DOMAIN, BLOCKS_PROBLEM_2)
  expected_F_sets = [{'(arm-empty)', '(clear c)', '(on-table a)', '(on c a)', '(clear b)', '(on-table b)'}, {'(clear c)', '(holding b)', '(on-table a)', '(holding c)', '(clear b)', '(clear a)', '(on-table b)', '(arm-empty)', '(on c a)'}, {'(clear c)', '(holding b)', '(on-table a)', '(holding c)', '(on b a)', '(on c c)', '(clear b)', '(holding a)', '(clear a)', '(on-table b)', '(arm-empty)', '(on b b)', '(on b c)', '(on c b)', '(on c a)', '(on-table c)'}, {'(holding b)', '(on-table a)', '(on c c)', '(clear a)', '(on-table b)', '(on b b)', '(on c a)', '(on a b)', '(clear c)', '(holding c)', '(on b a)', '(on a a)', '(clear b)', '(holding a)', '(on a c)', '(arm-empty)', '(on b c)', '(on c b)', '(on-table c)'}]
  F_sets, A_sets = construct_rpg(task)
  assert len(F_sets) == len(expected_F_sets), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
  for F_set, expected_F_set in zip(F_sets, expected_F_sets):
      assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
  expected_A_set_strs = [{'(unstack c a)', '(pick-up b)'}, {'(unstack c a)', '(pick-up b)', '(stack b a)', '(stack b c)', '(put-down c)', '(stack c a)', '(stack c c)', '(put-down b)', '(stack c b)', '(pick-up a)', '(stack b b)'}, {'(stack b a)', '(stack b c)', '(unstack b b)', '(stack c c)', '(stack c b)', '(unstack c b)', '(unstack b c)', '(stack a b)', '(put-down a)', '(stack a c)', '(pick-up a)', '(stack a a)', '(unstack c a)', '(pick-up b)', '(put-down c)', '(stack c a)', '(put-down b)', '(unstack b a)', '(stack b b)', '(pick-up c)', '(unstack c c)'}]
  assert len(A_sets) == len(expected_A_set_strs), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
  for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
      assert {a.name for a in A_set} == expected_A_set_str, f"Incorrect A_set: {A_set}"
  return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test2()
print('Tests passed.')

### Implement h_add
Given a Pyperplan Taks instance and a state (set of facts or None), return h_add(state).

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_add(task, state=None):
  """Computes h_add for a given Pyperplan Task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a pyperplan Task instance
    state: a set of facts

  Returns:
    h: an integer
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def h_add_test1(): return h_add_test(BLOCKS_PROBLEM_1, 3)

h_add_test1()
def h_add_test2(): return h_add_test(BLOCKS_PROBLEM_2, 5)

h_add_test2()
def h_add_test3(): return h_add_test(BLOCKS_PROBLEM_3, 21)

h_add_test3()
def h_add_test4(): return h_add_test(BLOCKS_PROBLEM_4, 30)

h_add_test4()
print('Tests passed.')

### Implement h_ff
Given a Pyperplan Taks instance and a state (set of facts or None), return h_ff(state).

For reference, our solution is **18** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_ff(task, state=None):
  """Computes h_ff for a given Pyperplan Task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a pyperplan Task instance
    state: a set of facts

  Returns:
    h: an integer
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def h_ff_test1(): return h_ff_test(BLOCKS_PROBLEM_1, 2)

h_ff_test1()
def h_ff_test2(): return h_ff_test(BLOCKS_PROBLEM_2, 5)

h_ff_test2()
def h_ff_test3(): return h_ff_test(BLOCKS_PROBLEM_3, 12)

h_ff_test3()
def h_ff_test4(): return h_ff_test(BLOCKS_PROBLEM_4, 16)

h_ff_test4()
print('Tests passed.')